In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

import warnings

In [2]:
from sklearn.linear_model import LassoCV, Lasso, SGDRegressor

Класс моделей ARIMA недостаточно богат для наших данных: с их помощью, например, никак нельзя учесть взаимосвязи между рядами. Это можно сделать с помощью векторной авторегрессии VARIMA, но её питоновская реализация не позволяет использовать регрессионные признаки. Кроме того, авторегрессионный подход не позволяет учитывать, например, взаимодействия между сезонными компонентами. Вы могли заметить, что форма суточных сезонных профилей в будни и выходные немного разная; явно моделировать этот эффект с помощью ARIMA не получится.

Нам нужна более сложная модель. Давайте займёмся сведением задачи массового прогнозирования рядов к регрессионной постановке!

Вам понадобится много признаков. Некоторые из них у вас уже есть — это:

* идентификатор географической зоны
* дата и время
* количество поездок в периоды, предшествующие прогнозируемому
* синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
* Кроме того, не спешите выбрасывать построенный вами на прошлой неделе прогнозы — из них может получиться хороший признак для регрессии!

Вы можете попробовать разные регрессионный модели, но хорошие результаты, скорее всего, дадут такие, которые будут позволять признакам взаимодействовать друг с другом.

Поскольку прогноз нужен на 6 часов вперёд, проще всего будет построить 6 независимых регрессионных моделей — одна для прогнозирования $\hat{y}_{T+1|T}$, другая для $\hat{y}_{T+2|T}$

Чтобы сдать задание, выполните следующую последовательность действий.

1. Для каждой из шести задач прогнозирования $\hat{y}_{T+i|T}, i=1,\dots,6 $ сформируйте выборки. Откликом будет $y_{T+i}$ при всевозможных значениях $T$, а признаки можно использовать следующие:


* идентификатор географической зоны — категориальный
* год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так
* синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
* сами значения прогнозов ARIMA $\hat{y}_{T+i|T}^{ARIMA}$ 
* количество поездок из рассматриваемого района в моменты времени $y_T, y_{T-1}, \dots, y_{T-K}$ (параметр $K$ можно подбирать; попробуйте начать, например, с 6)
* количество поездок из рассматриваемого района в моменты времени $y_{T-24}, y_{T-48}, \dots, y_{T-24*K_d}$ (параметр $K_d$ можно подбирать; попробуйте начать, например, с 2)
* суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц

Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании $\hat{y}_{T+i|T}, i=1,\dots,6$ вы можете учитывать только значения $y$ до момента времени $T$ включительно.

In [3]:
data = pd.read_csv('full_time_region.csv', index_col='tpep_pickup_datetime', parse_dates=True)

In [4]:
from os import listdir
from os.path import isfile, join
import re

files = [f for f in listdir('.') if isfile(join('.', f)) and f.startswith('time_region')]

data = []

for file in files:
    date = re.split('_|\.', file)[-2]
    time_reg = pd.read_csv('time_region_' + date + '.csv', index_col='tpep_pickup_datetime')
    data.append(time_reg)

names_of_regions = pd.read_csv('big_regions.csv').astype(str).values.reshape(-1)
data = pd.concat(data)
data = data[names_of_regions]
data.index = pd.to_datetime(data.index)

In [5]:
data.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
tpep_pickup_datetime,,,,,,,,,,,,,,,,,,,,,
2015-05-01 00:00:00,33,82,33,59,183,377,550,720,913,123,...,4,0,6,66,85,142,15,151,31,37
2015-05-01 01:00:00,18,39,21,59,115,240,310,443,808,89,...,14,0,1,1,2,15,6,90,23,32
2015-05-01 02:00:00,10,17,14,33,71,156,163,342,767,67,...,10,0,0,1,1,2,0,28,0,3
2015-05-01 03:00:00,9,10,7,8,44,103,118,207,657,62,...,14,0,0,0,0,0,0,2,1,9
2015-05-01 04:00:00,5,12,5,20,39,80,88,126,348,37,...,9,0,0,3,4,3,0,14,26,3


Подготовим основу для признаков. Тут уже есть регион (но в числовом виде) и $T$ как value

In [6]:
X = pd.DataFrame(data.stack(), columns=['T'])

X['datetime'] = X.reset_index().iloc[:, 0].values
X['region'] = X.reset_index().iloc[:, 1].values

 идентификатор географической зоны — категориальный
 
 год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так

In [7]:
X['region'] = X['region'].astype(str)

X['year'] = X['datetime'].dt.year
X['month'] = X['datetime'].dt.month
X['day'] = X['datetime'].dt.day
X['dayofweek'] = X['datetime'].dt.dayofweek
X['hour'] = X['datetime'].dt.hour

X['year_str'] = X['year'].astype(str)
X['month_str'] = X['month'].astype(str)
X['day_str'] = X['day'].astype(str)
X['dayofweek_str'] = X['dayofweek'].astype(str)
X['hour_str'] = X['hour'].astype(str)

X.drop('datetime', axis=1, inplace=True)

количество поездок из рассматриваемого района в моменты времени $y_T, y_{T-1}, \dots, y_{T-K}$ (параметр $K$ можно подбирать; попробуйте начать, например, с 6)

In [8]:
K = 6

preTs = data.shift(-1).dropna().stack()
preTs = pd.DataFrame(preTs, columns=['T-1'])

for i in range(2, K + 1):
    preT = data.shift(i).dropna().stack()
    preT = pd.DataFrame(preT, columns=[f'T-{i}'])
    preTs = preTs.merge(preT, left_index=True, right_index=True)

количество поездок из рассматриваемого района в моменты времени $y_{T-24}, y_{T-48}, \dots, y_{T-24*K_d}$ (параметр $K_d$ можно подбирать; попробуйте начать, например, с 2)

In [9]:
Kd = 2

preTds = data.shift(-1).dropna().stack()
preTds = pd.DataFrame(preTds, columns=['T-24'])

for i in range(2, Kd + 1):
    preTd = data.shift(i).dropna().stack()
    preTd = pd.DataFrame(preTd, columns=[f'T-{24 * i}'])
    preTds = preTds.merge(preTd, left_index=True, right_index=True)

суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц

In [10]:
halfday = data.rolling(6).sum().dropna().stack()
halfday = pd.DataFrame(halfday, columns=['halfday_sum'])
day = data.rolling(24).sum().dropna().stack()
day = pd.DataFrame(day, columns=['day_sum'])
week = data.rolling(24 * 7).sum().dropna().stack()
week = pd.DataFrame(week, columns=['week_sum'])
month = data.rolling(24 * 30).sum().dropna().stack()
month = pd.DataFrame(month, columns=['month_sum'])

sums = pd.merge(halfday, day, left_index=True, right_index=True)
sums = sums.merge(week, left_index=True, right_index=True)
sums = sums.merge(month, left_index=True, right_index=True)

Объединим имеющиеся признаки

In [11]:
X = X.merge(preTs, left_index=True, right_index=True)
X = X.merge(preTds, left_index=True, right_index=True)
X = X.merge(sums, left_index=True, right_index=True)

In [12]:
X.head()

T region  year  month  day  dayofweek  hour  \
tpep_pickup_datetime                                                       
2015-05-30 23:00:00  1075   91   1075  2015      5   30          5    23   
                     1076  189   1076  2015      5   30          5    23   
                     1077   92   1077  2015      5   30          5    23   
                     1125  124   1125  2015      5   30          5    23   
                     1126  366   1126  2015      5   30          5    23   

                          year_str month_str day_str  ...    T-3    T-4  \
tpep_pickup_datetime                                  ...                 
2015-05-30 23:00:00  1075     2015         5      30  ...  116.0  128.0   
                     1076     2015         5      30  ...  150.0  172.0   
                     1077     2015         5      30  ...  143.0  144.0   
                     1125     2015         5      30  ...  123.0  106.0   
                     1126     2015         5      30  ...  386.0  352.0   

                             T-5    T-6   T-24   T-48  halfday_sum  day_sum  \
tpep_pickup_datetime                                                          
2015-05-30 23:00:00  1075  161.0  157.0   63.0  109.0        700.0   1980.0   
                     1076  250.0  266.0  133.0  179.0       1122.0   3840.0   
                     1077  160.0  157.0   62.0  127.0        771.0   2337.0   
                     1125  131.0  109.0   93.0  117.0        693.0   1874.0   
                     1126  368.0  318.0  278.0  327.0       2183.0   5887.0   

                           week_sum  month_sum  
tpep_pickup_datetime                            
2015-05-30 23:00:00  1075   11941.0    50528.0  
                     1076   23396.0   102258.0  
                     1077   15230.0    67457.0  
                     1125   12156.0    55162.0  
                     1126   33189.0   148485.0  

[5 rows x 24 columns]

синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA

In [13]:
time = pd.DataFrame(index=X.index.levels[0])
number = 250
row = np.arange(1, X.index.levels[0].shape[0] + 1)
for i in range(1, number+1):
    time['s_year_'+str(i)] = np.sin(row*2*np.pi*i/8766)
    time['c_year_'+str(i)] = np.cos(row*2*np.pi*i/8766)
    time['s_week_'+str(i)] = np.sin(row*2*np.pi*i/168)
    time['c_week_'+str(i)] = np.cos(row*2*np.pi*i/168)

Соберем будущие значения на шесть часов вперед

In [14]:
ys = data.shift(-1).dropna().stack()
ys = pd.DataFrame(ys, columns=['T+1'])

for i in range(2, 7):
    y = data.shift(-i).dropna().stack()
    y = pd.DataFrame(y, columns=[f'T+{i}'])
    ys = ys.merge(y, left_index=True, right_index=True)

2. Разбейте каждую из шести выборок на три части:


* обучающая, на которой будут настраиваться параметры моделей — всё до май 2016
* тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
* итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016

In [15]:
april_X = X.loc[X.index.isin(X.index.levels[0][X.index.levels[0] < '2016-05-01 00:00:00'], level=0), :]
may_X = X.loc[X.index.isin(
    (X.index.levels[0][X.index.levels[0] >= '2016-05-01 00:00:00']) &
    (X.index.levels[0][X.index.levels[0] < '2016-06-01 00:00:00']),
    level=0), :]
june_X = X.loc[X.index.isin(
    (X.index.levels[0][X.index.levels[0] >= '2016-05-31 23:00:00']) &
    (X.index.levels[0][X.index.levels[0] <= '2016-06-30 17:00:00']),
    level=0), :]

In [16]:
april_time = time.loc[time.index[time.index < '2016-05-01 00:00:00'], :]
may_time = time.loc[time.index[
    (time.index >= '2016-05-01 00:00:00') &
    (time.index < '2016-06-01 00:00:00')
], :]
june_time = time.loc[time.index[
    (time.index >= '2016-05-31 23:00:00') &
    (time.index <= '2016-06-30 17:00:00')
], :]

Очень удобно в пандасе, что при мерджинге даже мультииндексы обрабатываются автоматически))

In [17]:
train = april_X.merge(april_time, left_index=True, right_index=True)
test = may_X.merge(may_time, left_index=True, right_index=True)
valid = june_X.merge(june_time, left_index=True, right_index=True)

In [18]:
ys = ys.loc[ys.index.isin(ys.index.levels[0][ys.index.levels[0] >= april_X.index.levels[0].min()], level=0), :]
train_ys = ys.loc[ys.index.isin(ys.index.levels[0][ys.index.levels[0] < '2016-05-01 00:00:00'], level=0), :]
test_ys = ys.loc[ys.index.isin(
    (ys.index.levels[0][ys.index.levels[0] >= '2016-05-01 00:00:00']) &
    (ys.index.levels[0][ys.index.levels[0] < '2016-06-01 00:00:00']),
    level=0), :]
valid_ys = ys.loc[ys.index.isin(
    (ys.index.levels[0][ys.index.levels[0] >= '2016-05-31 23:00:00']) &
    (ys.index.levels[0][ys.index.levels[0] <= '2016-06-30 17:00:00']),
    level=0), :]

3. Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:


* допускала попарные взаимодействия между признаками
* была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)

В ходе выполнения заданий за эту неделю, я столкнулся с серьезной проблемой слишком больших данных. Поэтому как простой вариант, я взял только нагенерированные признаки и на их основе Lasso построил. Даже с таким подходом ошибка уменьшилась

In [19]:
models = [Lasso().fit(april_X, train_ys[f'T+{i}'].values) for i in range(1, 7)]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1057255825.5962633, tolerance: 5153865.000252157
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2633419553.7985444, tolerance: 5153742.5031987345
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2874149380.6548967, tolerance: 5153328.431354502
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duali

In [20]:
# models = [Lasso().fit(april_time, train_ys[f'T+{i}'].values) for i in range(1, 7)]

4. Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:

$$
Q_{may}=\frac{1}{R*739*6}\sum_{r=1}^{R}{\sum_{T=2016.04.30\ 23:00}^{2016.05.31\ 17:00}{\sum_{i=1}^{6}{|\hat{y}^r_{T|T+i} - y^r_{T+i}|}}}
$$

Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу $Q$, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

In [21]:
predicts = np.array([model.predict(may_X) for model in models]).T
predicts[predicts < 0] = 0

In [22]:
np.mean(np.abs(test_ys.values - np.array(predicts)))

41.20619307898187

5. Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате *geoID, histEndDay, histEndHour, step, y*. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

In [23]:
valid_predict = pd.DataFrame(np.array([model.predict(june_X) for model in models]).T, index=valid_ys.index)
valid_predict = valid_predict.stack().reset_index()

In [24]:
def rewrite_month(date):
    return '0' + str(date.month) if date.month < 10 else str(date.month)

def rewrite_day(date):
    return '0' + str(date.day) if date.day < 10 else str(date.day)

In [25]:
reg_date = valid_predict['level_1'] + \
        '_' + valid_predict['tpep_pickup_datetime'].dt.year.astype(str) + \
        '-'+ valid_predict['tpep_pickup_datetime'].apply(rewrite_month) + \
        '-' + valid_predict['tpep_pickup_datetime'].apply(rewrite_day) + \
        '_' + valid_predict['tpep_pickup_datetime'].dt.hour.astype(str) + \
        '_' + (valid_predict['level_2'] + 1).astype(str)

In [26]:
valid_predict['id'] = reg_date
valid_predict = valid_predict[['id', 0]]
valid_predict.columns = ['id', 'y']

In [27]:
valid_predict

,id,y
0,1075_2016-05-31_23_1,24.808698
1,1075_2016-05-31_23_2,28.865819
2,1075_2016-05-31_23_3,34.408516
3,1075_2016-05-31_23_4,37.781713
4,1075_2016-05-31_23_5,39.797501
...,...,...
437575,2168_2016-06-30_17_2,4.099673
437576,2168_2016-06-30_17_3,6.414224
437577,2168_2016-06-30_17_4,6.070449
437578,2168_2016-06-30_17_5,3.131153


In [28]:
valid_predict.to_csv('to_kaggle_week5.csv', index=False)

6. Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

**Score: 39.89764**

7. Загрузите ноутбук в форму.

Done ^)

In [32]:
april_X.to_csv('april_X.csv')
may_X.to_csv('may_X.csv')
june_X.to_csv('june_X.csv')

train_ys.to_csv('train_ys.csv')
test_ys.to_csv('test_ys.csv')
valid_ys.to_csv('valid_ys.csv')